In [90]:
import pandas as pd
import numpy as np
import os

In [91]:
data_path = os.path.join(os.path.abspath(os.getcwd()), "..", "data")

df = pd.read_csv(os.path.join(data_path, "census.csv"))

df.head()

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Column names contain whitle spaces

In [92]:
df.columns

Index(['age', ' workclass', ' fnlgt', ' education', ' education-num',
       ' marital-status', ' occupation', ' relationship', ' race', ' sex',
       ' capital-gain', ' capital-loss', ' hours-per-week', ' native-country',
       ' salary'],
      dtype='object')

**df['workclass'] raises a KeyError**

## Remove spaces from column names

In [93]:
df.columns

Index(['age', ' workclass', ' fnlgt', ' education', ' education-num',
       ' marital-status', ' occupation', ' relationship', ' race', ' sex',
       ' capital-gain', ' capital-loss', ' hours-per-week', ' native-country',
       ' salary'],
      dtype='object')

In [94]:
df.columns = [col.strip() for col in df.columns]

df.columns

Index(['age', 'workclass', 'fnlgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'salary'],
      dtype='object')

In [95]:
df['workclass']

0                State-gov
1         Self-emp-not-inc
2                  Private
3                  Private
4                  Private
               ...        
32556              Private
32557              Private
32558              Private
32559              Private
32560         Self-emp-inc
Name: workclass, Length: 32561, dtype: object

## Save the clean data

In [7]:
data_path = os.path.join(os.path.abspath(os.getcwd()), "..", "data")

df.to_csv(os.path.join(data_path, "clean_census.csv"))

## Slice Performance

In [570]:
def compute_slice_metrics(
    features,
    labels,
    predictions,
    cat_features):
    
    labels = pd.Series(np.squeeze(labels))
    predictions = pd.Series(np.squeeze(predictions))
    
    # Construct the full dataframe containing labels and predictions   
    df = pd.concat([features, labels, predictions], axis=1)
    df.columns = list(features.columns) + ['labels', 'predictions']
    
    TP = df[df['labels'] == 1].groupby(cat_features)['predictions'].sum()
    FP = df[df['labels'] == 1].groupby(cat_features)['predictions'].apply(lambda x: x.count() - x.sum())
    TN = df[df['labels'] == 0].groupby(cat_features)['predictions'].apply(lambda x: x.count() - x.sum())
    FN = df[df['labels'] == 0].groupby(cat_features)['predictions'].sum()
    
    precision = (TP / (TP + FP))
    recall = (TP / (TP + FN))
    TNR = (TN / (TN + FP))
    NPV = (TN / (TN + FN))
    
    final_df = pd.concat([precision, recall, TNR, NPV], axis=1)
    final_df.columns = ['Precision', 'Recall', 'TNR', 'NPV']
    
    return final_df

In [572]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

labels = df['salary']
labels = lb.fit_transform(labels)

features = df.copy().drop(['salary'], axis=1)
cat_features = ['workclass']

slice_performance = compute_slice_metrics(
                        features,
                        labels,
                        labels,
                        cat_features)
        
print(slice_performance)

                   Precision  Recall  TNR  NPV
workclass                                     
 ?                       1.0     1.0  1.0  1.0
 Federal-gov             1.0     1.0  1.0  1.0
 Local-gov               1.0     1.0  1.0  1.0
 Private                 1.0     1.0  1.0  1.0
 Self-emp-inc            1.0     1.0  1.0  1.0
 Self-emp-not-inc        1.0     1.0  1.0  1.0
 State-gov               1.0     1.0  1.0  1.0
 Never-worked            NaN     NaN  NaN  1.0
 Without-pay             NaN     NaN  NaN  1.0
